In [2]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools
import os


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")



In [3]:
%matplotlib inline


In [4]:
spotify_df = pd.read_csv(r'/Users/sabaamiri/Documents/archive (2)/tracks.csv')


In [5]:
spotify_df.head()


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [6]:
data_w_genre = pd.read_csv(r'/Users/sabaamiri/Downloads/data_w_genres.csv')


In [7]:
data_w_genre.head()

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.575083,0.442750,247260.000000,0.386336,0.022717,0.287708,-14.205417,0.180675,115.983500,0.334433,38.000000,5,1,12,['show tunes']
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,33.076923,5,1,26,[]
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.285714,0,1,7,[]
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.444444,0,1,27,[]
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.605444,0.437333,232428.111111,0.429333,0.037534,0.216111,-11.447222,0.086000,120.329667,0.458667,42.555556,11,1,9,[]


In [8]:
data_w_genre.dtypes


artists              object
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
tempo               float64
valence             float64
popularity          float64
key                   int64
mode                  int64
count                 int64
genres               object
dtype: object

In [9]:
data_w_genre['genres_upd'] = data_w_genre['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])


In [10]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))


In [11]:
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))

In [12]:
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'] )


In [13]:
spotify_df['artists_song'] = spotify_df.apply(lambda row: str(row['artists_upd'][0])+ str(row['name']),axis = 1)


In [14]:
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)


In [15]:
spotify_df[spotify_df['name']=='Adore You']


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,...,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,...,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You
92524,1M4qEo4HE3PRaCOM7EXNJq,Adore You,74,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-06,0.676,0.771,...,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [16]:
spotify_df.drop_duplicates('artists_song',inplace = True)


In [17]:
spotify_df[spotify_df['name']=='Adore You']


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,...,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,...,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [22]:
spotify_df.tail(1000)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
76294,3ahDa9N1X8765hCa7AMJmR,"Onward, Christian Soldiers",1,381333,0,['2002 Philadelphia Singers'],['795Zjx3LdbkTug23uGtqUo'],2015-09-08,0.161,0.211,...,0.921,0.000191,0.1820,0.0624,83.845,4,[2002 Philadelphia Singers],[],[2002 Philadelphia Singers],"2002 Philadelphia SingersOnward, Christian Sol..."
76292,48uNq9y6RenX5MMXf2dt1Q,Faithful Warrior,0,407640,0,['2002 Philadelphia Singers'],['795Zjx3LdbkTug23uGtqUo'],2013-04-23,0.061,0.109,...,0.732,0.924000,0.0934,0.0380,60.265,3,[2002 Philadelphia Singers],[],[2002 Philadelphia Singers],2002 Philadelphia SingersFaithful Warrior
76290,5Z0n0UZemml67KgRmuIdsk,A New Day Dawns,1,308973,0,['2002 Philadelphia Singers'],['795Zjx3LdbkTug23uGtqUo'],2013-04-23,0.141,0.214,...,0.960,0.000104,0.0773,0.0941,82.996,4,[2002 Philadelphia Singers],[],[2002 Philadelphia Singers],2002 Philadelphia SingersA New Day Dawns
76296,61xP2ttiIdfiNHIozzLNdW,Prayer of Thanks,1,233507,0,['2002 Philadelphia Singers Women'],['3QYGdrYcO6SuWMyJ08UYtd'],2013-04-23,0.205,0.121,...,0.974,0.237000,0.1030,0.1010,150.641,4,[2002 Philadelphia Singers Women],[],[2002 Philadelphia Singers Women],2002 Philadelphia Singers WomenPrayer of Thanks
76298,0MNE8e1mtITNjxWRqSTbxM,Ya Better Not Run (feat. Conner Brown),1,174827,0,"['2002 Philadelphia Singers Men', 'Conner Brown']","['3xvjwd1JhU9eYjS58H5wNS', '4nbYt1QbydvtzANA2q...",2013-04-23,0.582,0.269,...,0.969,0.000000,0.3440,0.6740,115.564,4,"[2002 Philadelphia Singers Men, Conner Brown]",[],"[2002 Philadelphia Singers Men, Conner Brown]",2002 Philadelphia Singers MenYa Better Not Run...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158908,3YCC4oCzagcfx4s3fp3XFz,Bandolera,35,322926,1,"[""At' Fat"", 'Yemil', 'Boza', 'El Tachi']","['455JBMYIT1lWf1Djj4rDc7', '7g4cPtKxNx146qdaY9...",2020-11-17,0.502,0.814,...,0.524,0.000000,0.1510,0.6720,72.344,4,"[ Fat"", , , , , ]",[At' Fat],"[ Fat"", , , , , ]","Fat"", Bandolera"
537682,5sr38tHMSPpaENiNcjUpFi,Cathrina,10,184560,0,"[""Henry D' Cruz"", ""Helen D'Souza""]","['3nH2WLSi1MhvlmJNkbLbIv', '4NR8lI9YoPWo5v2Kv6...",1976-01-01,0.661,0.677,...,0.683,0.001540,0.0361,0.8090,116.855,4,"[ Cruz"", ""Helen D]","[Henry D' Cruz, Helen D'Souza]","[ Cruz"", ""Helen D]","Cruz"", ""Helen DCathrina"
281020,6Pu2leLWEfThVIqJilw6O9,The Passion of Our Lord according to St. Matth...,4,79027,0,"[""Boys' Choir of The Church of The Transfigura...","['47EwjzhfZKotjVtvlQFNFS', '5H155SiWINLkYRElqw...",1963,0.177,0.208,...,0.988,0.946000,0.0982,0.0587,132.053,3,"[ Choir of The Church of The Transfiguration"",...",[Boys' Choir of The Church of The Transfigurat...,"[ Choir of The Church of The Transfiguration"",...","Choir of The Church of The Transfiguration"", ..."
294380,7f092ECp06XbFS6Ms5Yk6R,Mia Oraia Petalouda,26,100313,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.773,0.183,...,0.956,0.000000,0.1190,0.8640,79.950,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenMia Oraia Petalouda


In [37]:
spotify_df[spotify_df['name']=='Lost']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
286801,4dWXUeSAwhgGHaoJkSJK2w,Lost,45,179720,0,"['Trista', 'Julinho Ksd']","['1hsrQJC4A7sVS6wOwTxRxw', '7kR1Yw4RqYhhDD3a8Q...",2021-03-26,0.628,0.485,...,0.374000,0.000628,0.1110,0.3100,106.966,4,"[Trista, Julinho Ksd]",[],"[Trista, Julinho Ksd]",TristaLost
491506,6HYAJgKfPlzaBgU2KB5tXX,Lost,33,212733,0,['The Badloves'],['64czVVta8TqMAqkZ0M0Avc'],1993,0.495,0.568,...,0.085800,0.000003,0.1790,0.5180,112.315,4,[The Badloves],[],[The Badloves],The BadlovesLost
71515,4LLHvfvE21JQFD9ZKVCbd0,Lost,67,288157,1,"['T-Dre', 'Delux', 'Cali Life Style']","['7wGbzHaUlDhx4dUO4PCSmD', '4FqhfuOE9gIZuYAVIq...",1998,0.762,0.545,...,0.211000,0.000006,0.1220,0.0390,93.629,4,"[T-Dre, Delux, Cali Life Style]",[],"[T-Dre, Delux, Cali Life Style]",T-DreLost
186864,3XvemAiv640r7xnX2M17PD,Lost,43,478587,0,['SIX60'],['1caoBfXJrbKCwIaTzGkyHn'],2011-01-01,0.338,0.155,...,0.215000,0.000023,0.1150,0.0566,129.779,4,[SIX60],[],[SIX60],SIX60Lost
356624,1bHYLphlNkp5rHNRm72rC1,Lost,39,351907,0,['Roger Sanchez'],['1HT9k1ZSUL9IczSstOAgWJ'],2006-06-27,0.703,0.743,...,0.022500,0.074000,0.1140,0.5910,128.016,4,[Roger Sanchez],[],[Roger Sanchez],Roger SanchezLost
316529,0EI5TXVsfOQysxzuoFUyWy,Lost,1,236333,0,['Pálmi Gunnarsson'],['2tGETO39QXKpmeQMSzr5I6'],1980-01-01,0.604,0.914,...,0.068900,0.000330,0.2130,0.8480,152.915,4,[Pálmi Gunnarsson],[],[Pálmi Gunnarsson],Pálmi GunnarssonLost
264319,5odScOdal7jhpQ5DK6NN1h,Lost,32,215184,1,['Pil C'],['2U1l35agymGrBDFeMIxVLG'],2016-03-26,0.926,0.552,...,0.118000,0.000000,0.0841,0.5110,114.007,4,[Pil C],[],[Pil C],Pil CLost
460747,7DymdkJxjqsegefUnJlFQx,Lost,40,203333,0,['Noir Désir'],['4ksCwAPgMi8rkQwwR3nMos'],2001-01-01,0.327,0.675,...,0.378000,0.000003,0.0935,0.3780,100.115,4,[Noir Désir],[],[Noir Désir],Noir DésirLost
313492,6dexp1WYGzQWDr8DA8yDHG,Lost,59,169667,0,['Noak Hellsing'],['47WlUGFrUj1hDhjzX1l6nc'],2019-11-22,0.665,0.314,...,0.688000,0.000000,0.2910,0.4930,83.934,4,[Noak Hellsing],[],[Noak Hellsing],Noak HellsingLost
81006,6vhYDNMZgffPwcdXdvMqCS,Lost,60,219840,0,['Michael Bublé'],['1GxkXlMwML1oSg5eLPiAz3'],2007-04-30,0.518,0.354,...,0.764000,0.000550,0.1140,0.3150,137.749,4,[Michael Bublé],[],[Michael Bublé],Michael BubléLost


In [38]:
spotify_df[spotify_df['name']=='Go Crazy']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
94051,7uDkaEYlGVTK3D5ZIPSL2Z,Go Crazy,49,228000,1,['Lil Tjay'],['6jGMq4yGs7aQzuGsMgVgZR'],2021-04-05,0.690,0.591,...,0.0168,0.000,0.126,0.234,106.105,4,[Lil Tjay],[],[Lil Tjay],Lil TjayGo Crazy
420181,3CZT4f6YKxs8oKszblac31,Go Crazy,34,212599,0,['Daio'],['4mMSOURBVBn2Mslg6qiLdL'],2018-08-16,0.606,0.748,...,0.0162,0.831,0.197,0.442,127.083,4,[Daio],[],[Daio],DaioGo Crazy
92989,1IIKrJVP1C9N7iPtG6eOsK,Go Crazy,85,176960,1,"['Chris Brown', 'Young Thug']","['7bXgB6jMjp9ATFy66eO08Z', '50co4Is1HCEo8bhOyU...",2020-05-08,0.755,0.578,...,0.3160,0.000,0.250,0.581,94.148,4,"[Chris Brown, Young Thug]",[],"[Chris Brown, Young Thug]",Chris BrownGo Crazy
